# Statistics


**IDEAS**
- SMETANA
    - Find likelyhood that a community member from family X is dependent on a community member in family Y.
    - Find likelyhood that a community member from family X is dependent on a compound A.

- SteadierCOM
    - No statistics?

In [358]:
import math

import pandas as pd
from molmass import Formula
import reframed

In [359]:
import sys
sys.path.append("../functions/")
import general_functions as general_func

import colors_MAGs
import scipy.stats as stats

In [360]:
import numpy as np

### Load universal model

In [361]:
model_uni = reframed.load_cbmodel("/Users/idunmariaburgos/universal_model_extension/output/universe_bacteria.xml")

### Load data 

In [362]:
all_mags_paper = general_func.read_allmags_data()

In [363]:
#SC1_C = pd.read_csv("../output/steadiercom_sample_0.1.3/results/results_99_SC1_C.tsv",sep="\t")
#SC2_C = pd.read_csv("../output/steadiercom_sample_0.1.3/results/results_99_SC2_C.tsv",sep="\t")
SC1_X = pd.read_csv("../output/steadiercom_sample_0.1.3/results/results_99_SC1_X.tsv",sep="\t")

steadier_sample = pd.concat([SC1_X])
steadier_sample = steadier_sample[(steadier_sample.frequency>0.1) & (steadier_sample.rate>1e-6)]

In [364]:
steadier_sample_cross = steadier_sample[(steadier_sample.donor!="environment") & (steadier_sample.receiver!="environment") ].copy()

### Prepare to process data

In [365]:
chebi_lut, chebi_interesting, chebi_colors_ser = colors_MAGs.chebi_rxn_color_func(rxn_based=False)

**Change names of family for readability and better grouping**

In [366]:
all_mags_paper_reduced = all_mags_paper[all_mags_paper.Substrate=="Xylan"].copy()

all_mags_paper_reduced[all_mags_paper_reduced["new_coverage"]>10][["Source","Substrate","Family","new_coverage"]].sort_values(["Source","Substrate"])

all_mags_paper_reduced["Family"] = all_mags_paper.apply(lambda row: "f_"+row.Family,axis=1)

total_members_family = all_mags_paper_reduced.groupby("Family").count()["Source"].to_dict()
all_mags_paper_reduced["Family"] = all_mags_paper_reduced.apply(lambda row: row.Family if total_members_family[row.Family]>1 else "Other",axis=1)


In [367]:
def mag2family(all_mags_paper):  
    
    family_groups = all_mags_paper.groupby("Family").groups
    mag2family_dict = {mag:family for family,mags in family_groups.items() for mag in mags}
    
    return family_groups,mag2family_dict

**Create dictionaries to translate components into larger groups - MAG-> family, compound-> super_class**

In [368]:
MAGs_steady_com = set(list(steadier_sample_cross.donor.values)+list(steadier_sample_cross.receiver.values))

MAG2sour_sub_id = pd.read_csv("../output/MAG2community_id.tsv",sep="\t",header=None)
MAG2sour_sub_id.columns=["MAG","community_id"]

met2superclass_dict = pd.read_csv("../output/met_chebi_class.tsv",sep="\t",index_col=0)["self defined super class"].to_dict()

family_groups,mag2family_dict = mag2family(all_mags_paper_reduced)

In [369]:
def family_donor(row):
    if row.donor=="environment":
        return "environment"
    else:
        return mag2family_dict[row.donor]


def family_receiver(row):
    if row.receiver=="environment":
        return "environment"
    else:
        return mag2family_dict[row.receiver]


In [370]:
def met2metname(met):
    met_name = model_uni.metabolites[met].name
    return met_name

### Process data

**All compounds**

In [371]:
steadier_sample.loc[:,"family_donor"] = steadier_sample.apply(family_donor,axis=1).copy()
steadier_sample.loc[:,"family_receiver"] = steadier_sample.apply(family_receiver,axis=1).copy()
steadier_sample = steadier_sample[steadier_sample.compound.isin(met2superclass_dict.keys())].copy()
steadier_sample.loc[:,"super_class"] = steadier_sample.apply(lambda x: met2superclass_dict[x.compound],axis=1)

compounds = steadier_sample["compound"].map(met2metname)
steadier_sample.drop("compound",axis=1,inplace=True)
steadier_sample.loc[:,"compound"] = compounds


**Compounds cross-fed**

In [372]:
steadier_sample_cross.loc[:,"family_donor"] = steadier_sample_cross.apply(family_donor,axis=1).copy()
steadier_sample_cross.loc[:,"family_receiver"] = steadier_sample_cross.apply(family_receiver,axis=1).copy()
steadier_sample_cross = steadier_sample_cross[steadier_sample_cross.compound.isin(met2superclass_dict.keys())].copy()
steadier_sample_cross.loc[:,"super_class"] = steadier_sample_cross.apply(lambda x: met2superclass_dict[x.compound],axis=1)

compounds = steadier_sample_cross["compound"].map(met2metname)
steadier_sample_cross.drop("compound",axis=1,inplace=True)
steadier_sample_cross.loc[:,"compound"] = compounds

# FILTERS OUT UNINTERESTING COMPOUNDS
#steadier_sample_cross = steadier_sample_cross[steadier_sample_cross.super_class.isin(chebi_lut.keys())].copy()

steadier_sample_cross["mass_rate*frequency"]=steadier_sample_cross["mass_rate"]*steadier_sample_cross["frequency"]

In [373]:
steadier_sample_cross

,donor,receiver,mass_rate,rate,frequency,community,medium,family_donor,family_receiver,super_class,compound,mass_rate*frequency
4,CH9-bin.6,CH9-bin.0,4.915243e-02,0.327400,1.00,CM_X,SC1_X,Other,Other,simple sugars,D-Xylose,4.915243e-02
11,CH9-bin.0,CH9-bin.6,1.477037e-02,0.098384,1.00,CM_X,SC1_X,Other,Other,simple sugars,D-Ribose,1.477037e-02
16,CH3-bin.2,CH3-bin.1,1.159098e-02,0.077207,0.80,M_X,SC1_X,f_Clostridiaceae,f_Clostridiaceae,simple sugars,D-Xylose,9.272784e-03
30,CH14-bin.1,CH14-bin.2,4.777875e-03,0.108459,0.24,CD_X,SC1_X,f_Lachnospiraceae,f_Lachnospiraceae,alcohols and aldehydes,Acetaldehyde,1.146690e-03
32,CH14-bin.1,CH14-bin.0,4.675429e-03,0.031143,0.79,CD_X,SC1_X,f_Lachnospiraceae,Other,simple sugars,D-Xylose,3.693589e-03
...,...,...,...,...,...,...,...,...,...,...,...,...
345,CH3-bin.2,CH3-bin.0,4.612004e-07,0.000004,0.33,M_X,SC1_X,f_Clostridiaceae,Other,nucleotides and derivatives,Thymine C5H6N2O2,1.521961e-07
347,CH14-bin.1,CH14-bin.4,4.147959e-07,0.000206,0.18,CD_X,SC1_X,f_Lachnospiraceae,Other,gases,Hydrogen,7.466326e-08
351,CH14-bin.4,CH14-bin.2,2.743474e-07,0.000008,0.73,CD_X,SC1_X,Other,f_Lachnospiraceae,gases,Hydrogen sulfide,2.002736e-07
353,CH9-bin.2,CH9-bin.5,2.537980e-07,0.000007,0.22,CM_X,SC1_X,f_Clostridiaceae,f_Lactobacillaceae,gases,Hydrogen sulfide,5.583556e-08


### Overview of groups

In [374]:
pd.Series({family:len(mags) for family,mags in family_groups.items()})

Other                 5
f_Clostridiaceae      3
f_Lachnospiraceae     3
f_Lactobacillaceae    2
dtype: int64

In [375]:
all_mags_paper_reduced[all_mags_paper_reduced["new_coverage"]>10][["Source","Substrate","Family","Genus","new_coverage"]].sort_values(["Source","Substrate"])

,Source,Substrate,Family,Genus,new_coverage
MAG,,,,,
CH14-bin.1,Compost_Digestate,Xylan,f_Lachnospiraceae,Lacrimispora,54.707294
CH14-bin.2,Compost_Digestate,Xylan,f_Lachnospiraceae,Robinsoniella,18.831114
CH14-bin.0,Compost_Digestate,Xylan,Other,Caproicproducens,15.617491
CH9-bin.0,Cow_Manure,Xylan,Other,Bifidobacterium,70.451904
CH3-bin.2,Marshland,Xylan,f_Clostridiaceae,Clostridium,74.153410
CH3-bin.1,Marshland,Xylan,f_Clostridiaceae,Clostridium_S,21.418886


### Functions for statistics

In [376]:
def find_non_dependent(row,metric):
    return len(family_groups[row.name[0]]) - row[metric] #row.flux_mg


def statistics_adjustments(statistics_df):
    
    statistics_df = statistics_df.sort_values(by="p_value").copy()
    statistics_df["i"] = statistics_df["p_value"].rank(method="max")
    statistics_df["p_value_benjamini_h"] = statistics_df.apply(lambda row: min(row.p_value*statistics_df.shape[0]/row.i,1),axis=1)
    statistics_df.sort_index(inplace=True)
    return statistics_df


def statistics_function(steadier_sample_cross,dependent_variable,independent_variable,metric="flux_mg",metric_thresh=1e-6,pvalue_thresh=0.1):

    # Get average of each family according to each possible value of the independent variable
    # dependent_variable,dependent_variable.split("_")[1] here it decides if it is in the receiver or in the donor (dependent_variable.split("_")[1]) and groups by the [family_receiver,receiver,compound] and takes the mean of this
    steadiercom_crossfeeding_donor = steadier_sample_cross.loc[:,[dependent_variable,dependent_variable.split("_")[1],independent_variable,metric]].groupby([dependent_variable,dependent_variable.split("_")[1],independent_variable]).mean().copy()
    dependent = steadiercom_crossfeeding_donor[steadiercom_crossfeeding_donor[metric]>metric_thresh].reset_index().groupby([dependent_variable,independent_variable]).count().copy()
    not_dependent = dependent.apply(find_non_dependent,metric=metric,axis=1)

    # Add data for the missing values
    all_categories =set(not_dependent.index.get_level_values(1))

    for family in dependent.index.get_level_values(0):
        for category in all_categories-set(not_dependent.xs(family).index):
            not_dependent[(family,category)]=len(family_groups[family]) 
            
    concat_df = pd.concat({"dependent":dependent[metric],"not_dependent":not_dependent},axis=1).fillna(0)


    statistics = {}
    for independent_var in set(concat_df.index.get_level_values(1)):
        # Get the sub_df for the super class
        concat_df_sub = concat_df.xs(independent_var,level=1).copy()

        statistics[independent_var] = {}

        # For each row (each family)
        for i,row in concat_df_sub.iterrows():
            
            # Get the data for all other family
            other = concat_df_sub.loc[concat_df_sub.index[concat_df_sub.index!=i],:]
            
            data = pd.DataFrame({i:row,"other":other.sum()}).transpose().to_numpy()
            statistics[independent_var][(i,"data")]= data
            
            
            # Get odds ratio
            odds_ratio_num = data[0][0]
            odds_ratio_den = data[0][0] + data[0][1]
            other_num = data[1][0]
            other_den = data[1][0] + data[1][1]
            
            if odds_ratio_den == 0 or other_den == 0 or other_num==0:
                odds_ratio = math.inf
            else:
                odds_ratio = (odds_ratio_num / odds_ratio_den) / (other_num / other_den)
            statistics[independent_var][(i, "odds_ratio")] = odds_ratio
            
            
            # Calculate the Barnard exact statistical 
            p_value = stats.barnard_exact(pd.DataFrame({i:row,"other":other.sum()}).transpose().to_numpy(),alternative="greater")
            statistics[independent_var][(i,"p_value")]= p_value.pvalue
            

    statistics_df = pd.DataFrame(statistics)
    

    category_values = statistics_df.xs('p_value', level=1)
    

    values = {}
    for family,independent_var in category_values[category_values[category_values<pvalue_thresh].notnull()].stack().index:
        values[family,independent_var]= {"p_value":statistics_df.loc[(family,"p_value"),independent_var],"odds_ratio":statistics_df.loc[(family,"odds_ratio"),independent_var],"data":statistics_df.loc[(family,"data"),independent_var],"# interaction":concat_df.loc[(family,independent_var),"dependent"],"# no interaction":concat_df.loc[(family,independent_var),"not_dependent"]}

    significant = pd.DataFrame(values).transpose()
    if significant.empty==False:
        significant.index.names = (dependent_variable,independent_variable)

    return significant

## Likelyhood that family X is receiving compound A

In [377]:
statistics_df = statistics_function(steadier_sample_cross,"family_receiver","compound",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(48, 7)

In [378]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_receiver,compound,,,,,,,


In [379]:
statistics_df = statistics_function(steadier_sample_cross,"family_receiver","super_class",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(16, 7)

In [380]:
statistics_df[(statistics_df.p_value_benjamini_h<0.1) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_receiver,super_class,,,,,,,


## Likelyhood that family X is donating a compound A to another family

In [381]:
statistics_df = statistics_function(steadier_sample_cross,"family_donor","compound",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(47, 7)

In [382]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_donor,compound,,,,,,,


In [383]:
statistics_df = statistics_function(steadier_sample_cross,"family_donor","super_class",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(14, 7)

In [384]:
statistics_df[(statistics_df.p_value_benjamini_h<0.1) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_donor,super_class,,,,,,,


## Likelyhood that family X is receiving from family Y

In [385]:
statistics_df = statistics_function(steadier_sample_cross,"family_receiver","family_donor",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(8, 7)

In [386]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_receiver,family_donor,,,,,,,


### Who donates the most?

In [387]:
total_members = steadier_sample_cross.groupby("family_donor").nunique()["donor"]

### All communities - Relative abundance (above 10 %)

In [388]:
def abundance_statistic(abundance_communities,receiver_or_donor="receiver"):
    
    
    table_abundance_rec_don_dict = {}
    table_not_abundant_rec_don_dict = {}

    for compound in abundance_communities[abundance_communities.receiver_abundance_10].compound.unique():
        if receiver_or_donor=="receiver":
            abund_rec_don = len(abundance_communities[(abundance_communities.receiver_abundance_10) & (abundance_communities.compound==compound)].receiver.unique())
            not_abund_rec_don = len(abundance_communities[(~abundance_communities.receiver_abundance_10) & (abundance_communities.compound==compound)].receiver.unique())
        elif receiver_or_donor=="donor":
            abund_rec_don = len(abundance_communities[(abundance_communities.donor_abundance_10) & (abundance_communities.compound==compound)].donor.unique())
            not_abund_rec_don = len(abundance_communities[(~abundance_communities.donor_abundance_10) & (abundance_communities.compound==compound)].donor.unique())
        
        
        abund_not_rec_don = len(high_abundance) - abund_rec_don
        not_abund_not_rec_don = len(low_abunance) - not_abund_rec_don


        table = [[abund_rec_don,not_abund_rec_don],[abund_not_rec_don,not_abund_not_rec_don]]
        table_non_abundant = [[not_abund_rec_don,abund_rec_don],[not_abund_not_rec_don,abund_not_rec_don]]

        try:
            odds_ratio = (abund_rec_don/(abund_rec_don+abund_not_rec_don))/(not_abund_rec_don/(not_abund_rec_don+not_abund_not_rec_don))
        except:
            odds_ratio = math.inf


        p_value = stats.barnard_exact(table,alternative="greater")

        table_abundance_rec_don_dict[compound] = {"table":table,"p_value":p_value.pvalue,"odds_ratio":odds_ratio}


        p_value_not = stats.barnard_exact(table_non_abundant,alternative="greater")

        table_not_abundant_rec_don_dict[compound]= {"table":table,"p_value":p_value_not.pvalue,"odds_ratio":odds_ratio}
        
    return table_abundance_rec_don_dict, table_not_abundant_rec_don_dict
        


**Assign abundant/not-abundant classifier**

In [389]:
abundance_communities = steadier_sample_cross.copy()
abundance_communities["donor_abundance_10"] = abundance_communities["donor"].map(lambda x: all_mags_paper.loc[x,"new_coverage"]>10)
abundance_communities["receiver_abundance_10"] = abundance_communities["receiver"].map(lambda x: all_mags_paper.loc[x,"new_coverage"]>10)

**Overview of abundant members**

In [390]:
high_abundance = set(list(abundance_communities[abundance_communities.donor_abundance_10].donor.unique()) + list(abundance_communities[abundance_communities.receiver_abundance_10].receiver.unique()))
low_abunance = set(list(abundance_communities[~abundance_communities.donor_abundance_10].donor.unique()) + list(abundance_communities[~abundance_communities.receiver_abundance_10].receiver.unique()))

#### Statistics for receiving

In [391]:
table_abundance_rec_dict, table_not_abundant_rec_dict = abundance_statistic(abundance_communities,receiver_or_donor="receiver")

In [392]:
abund_rec_df = pd.DataFrame(table_abundance_rec_dict).T
abund_rec_df = statistics_adjustments(abund_rec_df)

abund_rec_df[abund_rec_df.p_value_benjamini_h<0.05].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


In [393]:
not_abund_rec_df = pd.DataFrame(table_not_abundant_rec_dict).T
not_abund_rec_df = statistics_adjustments(not_abund_rec_df)
not_abund_rec_df["odds_ratio"] = not_abund_rec_df["odds_ratio"].map(lambda x: 1/x)
not_abund_rec_df[not_abund_rec_df.p_value_benjamini_h<0.05].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


#### Statistics for donating

In [394]:
table_abundant_donation_dict, table_not_abundant_donation_dict = abundance_statistic(abundance_communities,receiver_or_donor="donor")

In [395]:
abundance_don_df = pd.DataFrame(table_abundant_donation_dict).T
abundance_don_df = statistics_adjustments(abundance_don_df)
abundance_don_df[abundance_don_df.p_value_benjamini_h<0.05].sort_values("p_value")


,table,p_value,odds_ratio,i,p_value_benjamini_h


In [396]:
not_abundant_don_df = pd.DataFrame(table_not_abundant_donation_dict).T
not_abundant_don_df = statistics_adjustments(not_abundant_don_df)


not_abundant_don_df["odds_ratio"] = not_abundant_don_df["odds_ratio"].map(lambda x: math.inf if x==0 else 1/x)
not_abundant_don_df[not_abundant_don_df.p_value_benjamini_h<0.05].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


## Mass flow

#### Glucose

**Receiver of Glucose**

In [397]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glucose")].value_counts("family_receiver")

Series([], dtype: int64)

In [398]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glucose")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


Series([], Name: mass_rate*frequency, dtype: float64)

**Donor of Glucose**

In [399]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glucose")].value_counts("family_donor")

Series([], dtype: int64)

In [400]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glucose")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


Series([], Name: mass_rate*frequency, dtype: float64)

In [401]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glucose")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


Series([], Name: mass_rate*frequency, dtype: float64)

In [402]:
steadier_sample_cross[(steadier_sample_cross.compound=="D-Glucose")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


Series([], Name: mass_rate*frequency, dtype: float64)

#### Acetaldehyde

**Receiver of Acetaldehyde**

In [403]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetaldehyde")].value_counts("family_receiver")

family_receiver
Other                 6
f_Lactobacillaceae    4
f_Lachnospiraceae     3
f_Clostridiaceae      2
dtype: int64

In [404]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetaldehyde")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
Other                 0.006709
f_Lactobacillaceae    0.003790
f_Lachnospiraceae     0.002421
f_Clostridiaceae      0.001012
Name: mass_rate*frequency, dtype: float64

**Donor of Acetaldehyde**

In [405]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetaldehyde")].value_counts("family_donor")

family_donor
f_Clostridiaceae     8
Other                4
f_Lachnospiraceae    3
dtype: int64

In [406]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetaldehyde")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
Other                0.008201
f_Clostridiaceae     0.004054
f_Lachnospiraceae    0.001676
Name: mass_rate*frequency, dtype: float64

In [407]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetaldehyde")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver   
CD_X       f_Lachnospiraceae     0.001147
           Other                 0.000530
CM_X       Other                 0.006025
           f_Lactobacillaceae    0.003790
           f_Lachnospiraceae     0.001274
M_X        f_Clostridiaceae      0.001012
           Other                 0.000153
Name: mass_rate*frequency, dtype: float64

In [408]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetaldehyde")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor     
CD_X       f_Lachnospiraceae    0.001676
CM_X       Other                0.008201
           f_Clostridiaceae     0.002889
M_X        f_Clostridiaceae     0.001165
Name: mass_rate*frequency, dtype: float64

#### H2S

**Receiver of H2S**

In [409]:
steadier_sample_cross[(steadier_sample_cross.compound=="Hydrogen sulfide")].value_counts("family_receiver")

family_receiver
f_Lactobacillaceae    6
f_Lachnospiraceae     5
f_Clostridiaceae      4
Other                 3
dtype: int64

In [410]:
steadier_sample_cross[(steadier_sample_cross.compound=="Hydrogen sulfide")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Clostridiaceae      0.000044
f_Lachnospiraceae     0.000014
Other                 0.000008
f_Lactobacillaceae    0.000004
Name: mass_rate*frequency, dtype: float64

**Donor of H2S**

In [411]:
steadier_sample_cross[(steadier_sample_cross.compound=="Hydrogen sulfide")].value_counts("family_donor")

family_donor
Other                9
f_Lachnospiraceae    5
f_Clostridiaceae     4
dtype: int64

In [412]:
steadier_sample_cross[(steadier_sample_cross.compound=="Hydrogen sulfide")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Clostridiaceae     0.000039
Other                0.000020
f_Lachnospiraceae    0.000012
Name: mass_rate*frequency, dtype: float64

In [413]:
steadier_sample_cross[(steadier_sample_cross.compound=="Hydrogen sulfide")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver   
CD_X       f_Lachnospiraceae     0.000014
CM_X       Other                 0.000008
           f_Clostridiaceae      0.000005
           f_Lactobacillaceae    0.000004
M_X        f_Clostridiaceae      0.000039
Name: mass_rate*frequency, dtype: float64

In [414]:
steadier_sample_cross[(steadier_sample_cross.compound=="Hydrogen sulfide")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor     
CD_X       Other                1.156144e-05
           f_Lachnospiraceae    2.336506e-06
CM_X       f_Lachnospiraceae    9.185359e-06
           Other                7.683189e-06
           f_Clostridiaceae     2.758023e-07
M_X        f_Clostridiaceae     3.860965e-05
           Other                6.359372e-07
Name: mass_rate*frequency, dtype: float64

#### CO2

**Receiver of CO2**

In [415]:
steadier_sample_cross[(steadier_sample_cross.compound=="CO2 CO2")].value_counts("family_receiver")

family_receiver
f_Lachnospiraceae     8
f_Lactobacillaceae    5
dtype: int64

In [416]:
steadier_sample_cross[(steadier_sample_cross.compound=="CO2 CO2")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Lachnospiraceae     0.001586
f_Lactobacillaceae    0.000037
Name: mass_rate*frequency, dtype: float64

**Donor of CO2**

In [417]:
steadier_sample_cross[(steadier_sample_cross.compound=="CO2 CO2")].value_counts("family_donor")

family_donor
Other                 6
f_Lactobacillaceae    3
f_Clostridiaceae      2
f_Lachnospiraceae     2
dtype: int64

In [418]:
steadier_sample_cross[(steadier_sample_cross.compound=="CO2 CO2")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Lachnospiraceae     1.157005e-03
Other                 4.597303e-04
f_Lactobacillaceae    5.622164e-06
f_Clostridiaceae      8.188731e-07
Name: mass_rate*frequency, dtype: float64

In [419]:
steadier_sample_cross[(steadier_sample_cross.compound=="CO2 CO2")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver   
CD_X       f_Lachnospiraceae     0.001411
CM_X       f_Lachnospiraceae     0.000175
           f_Lactobacillaceae    0.000037
Name: mass_rate*frequency, dtype: float64

In [420]:
steadier_sample_cross[(steadier_sample_cross.compound=="CO2 CO2")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor      
CD_X       f_Lachnospiraceae     1.156611e-03
           Other                 2.547838e-04
CM_X       Other                 2.049465e-04
           f_Lactobacillaceae    5.622164e-06
           f_Clostridiaceae      8.188731e-07
           f_Lachnospiraceae     3.932263e-07
Name: mass_rate*frequency, dtype: float64

#### Lactate

**Receiver of L-Lactate**

In [421]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lactate")].value_counts("family_receiver")

family_receiver
Other               2
f_Clostridiaceae    1
dtype: int64

In [422]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lactate")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Clostridiaceae    0.001763
Other               0.000039
Name: mass_rate*frequency, dtype: float64

**Donor of L-Lactate**

In [423]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lactate")].value_counts("family_donor")

family_donor
Other               2
f_Clostridiaceae    1
dtype: int64

In [424]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lactate")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Clostridiaceae    0.001763
Other               0.000039
Name: mass_rate*frequency, dtype: float64

In [425]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lactate")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver 
CD_X       Other               0.000039
M_X        f_Clostridiaceae    0.001763
Name: mass_rate*frequency, dtype: float64

In [426]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lactate")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor    
CD_X       Other               0.000039
M_X        f_Clostridiaceae    0.001763
Name: mass_rate*frequency, dtype: float64

#### Guanine

**Receiver of Guanine**

In [427]:
steadier_sample_cross[(steadier_sample_cross.compound=="Guanine")].value_counts("family_receiver")

family_receiver
Other                 5
f_Clostridiaceae      2
f_Lactobacillaceae    2
f_Lachnospiraceae     1
dtype: int64

In [428]:
steadier_sample_cross[(steadier_sample_cross.compound=="Guanine")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
Other                 0.000363
f_Clostridiaceae      0.000086
f_Lachnospiraceae     0.000067
f_Lactobacillaceae    0.000027
Name: mass_rate*frequency, dtype: float64

**Donor of Guanine**

In [429]:
steadier_sample_cross[(steadier_sample_cross.compound=="Guanine")].value_counts("family_donor")

family_donor
f_Lachnospiraceae    8
f_Clostridiaceae     2
dtype: int64

In [430]:
steadier_sample_cross[(steadier_sample_cross.compound=="Guanine")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Lachnospiraceae    0.000465
f_Clostridiaceae     0.000077
Name: mass_rate*frequency, dtype: float64

In [431]:
steadier_sample_cross[(steadier_sample_cross.compound=="Guanine")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver   
CD_X       f_Lachnospiraceae     0.000067
           Other                 0.000065
CM_X       Other                 0.000292
           f_Lactobacillaceae    0.000027
           f_Clostridiaceae      0.000014
M_X        f_Clostridiaceae      0.000072
           Other                 0.000005
Name: mass_rate*frequency, dtype: float64

In [432]:
steadier_sample_cross[(steadier_sample_cross.compound=="Guanine")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor     
CD_X       f_Lachnospiraceae    0.000132
CM_X       f_Lachnospiraceae    0.000333
M_X        f_Clostridiaceae     0.000077
Name: mass_rate*frequency, dtype: float64

#### Uracil

**Receiver of Uracil**

In [433]:
steadier_sample_cross[(steadier_sample_cross.compound=="Uracil")].value_counts("family_receiver")

family_receiver
Other                 6
f_Lachnospiraceae     3
f_Clostridiaceae      2
f_Lactobacillaceae    2
dtype: int64

In [434]:
steadier_sample_cross[(steadier_sample_cross.compound=="Uracil")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
Other                 0.000162
f_Lachnospiraceae     0.000090
f_Clostridiaceae      0.000080
f_Lactobacillaceae    0.000012
Name: mass_rate*frequency, dtype: float64

**Donor of Uracil**

In [435]:
steadier_sample_cross[(steadier_sample_cross.compound=="Uracil")].value_counts("family_donor")

family_donor
f_Lachnospiraceae    8
Other                3
f_Clostridiaceae     2
dtype: int64

In [436]:
steadier_sample_cross[(steadier_sample_cross.compound=="Uracil")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Lachnospiraceae    0.000226
f_Clostridiaceae     0.000078
Other                0.000041
Name: mass_rate*frequency, dtype: float64

In [437]:
steadier_sample_cross[(steadier_sample_cross.compound=="Uracil")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver   
CD_X       f_Lachnospiraceae     0.000090
           Other                 0.000033
CM_X       Other                 0.000124
           f_Lactobacillaceae    0.000012
           f_Clostridiaceae      0.000008
M_X        f_Clostridiaceae      0.000073
           Other                 0.000005
Name: mass_rate*frequency, dtype: float64

In [438]:
steadier_sample_cross[(steadier_sample_cross.compound=="Uracil")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor     
CD_X       f_Lachnospiraceae    0.000082
           Other                0.000041
CM_X       f_Lachnospiraceae    0.000144
M_X        f_Clostridiaceae     0.000078
Name: mass_rate*frequency, dtype: float64

#### L-Aspartate

**Receiver of L-Aspartate**

In [439]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Aspartate")].value_counts("family_receiver")

Series([], dtype: int64)

In [440]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Aspartate")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


Series([], Name: mass_rate*frequency, dtype: float64)

**Donor of L-Aspartate**

In [441]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Aspartate")].value_counts("family_donor")

Series([], dtype: int64)

In [442]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Aspartate")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


Series([], Name: mass_rate*frequency, dtype: float64)

In [443]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Aspartate")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


Series([], Name: mass_rate*frequency, dtype: float64)

In [444]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Aspartate")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


Series([], Name: mass_rate*frequency, dtype: float64)

#### L-Lysine

**Receiver of L-Lysine**

In [445]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lysine")].value_counts("family_receiver")

family_receiver
f_Clostridiaceae      2
f_Lactobacillaceae    2
Other                 1
dtype: int64

In [446]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lysine")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
Other                 0.000167
f_Lactobacillaceae    0.000012
f_Clostridiaceae      0.000010
Name: mass_rate*frequency, dtype: float64

**Donor of L-Lysine**

In [447]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lysine")].value_counts("family_donor")

family_donor
f_Lactobacillaceae    5
dtype: int64

In [448]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lysine")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Lactobacillaceae    0.000189
Name: mass_rate*frequency, dtype: float64

In [449]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lysine")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver   
CM_X       Other                 0.000167
           f_Lactobacillaceae    0.000012
           f_Clostridiaceae      0.000010
Name: mass_rate*frequency, dtype: float64

In [450]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Lysine")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor      
CM_X       f_Lactobacillaceae    0.000189
Name: mass_rate*frequency, dtype: float64

#### L-Arginine

**Receiver of L-Arginine**

In [451]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Arginine")].value_counts("family_receiver")

family_receiver
Other                 4
f_Clostridiaceae      2
f_Lactobacillaceae    2
dtype: int64

In [452]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Arginine")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
Other                 0.000408
f_Clostridiaceae      0.000348
f_Lactobacillaceae    0.000021
Name: mass_rate*frequency, dtype: float64

**Donor of L-Arginine**

In [453]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Arginine")].value_counts("family_donor")

family_donor
f_Lactobacillaceae    8
dtype: int64

In [454]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Arginine")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Lactobacillaceae    0.000777
Name: mass_rate*frequency, dtype: float64

In [455]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Arginine")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver   
CM_X       Other                 0.000408
           f_Clostridiaceae      0.000348
           f_Lactobacillaceae    0.000021
Name: mass_rate*frequency, dtype: float64

In [456]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Arginine")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor      
CM_X       f_Lactobacillaceae    0.000777
Name: mass_rate*frequency, dtype: float64

#### Thymine C5H6N2O2

**Receiver of Thymine C5H6N2O2**

In [457]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].value_counts("family_receiver")

family_receiver
Other                1
f_Lachnospiraceae    1
dtype: int64

In [458]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Lachnospiraceae    1.636116e-06
Other                1.521961e-07
Name: mass_rate*frequency, dtype: float64

**Donor of Thymine C5H6N2O2**

In [459]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].value_counts("family_donor")

family_donor
Other               1
f_Clostridiaceae    1
dtype: int64

In [460]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
Other               1.636116e-06
f_Clostridiaceae    1.521961e-07
Name: mass_rate*frequency, dtype: float64

In [461]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver  
CD_X       f_Lachnospiraceae    1.636116e-06
M_X        Other                1.521961e-07
Name: mass_rate*frequency, dtype: float64

In [462]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor    
CD_X       Other               1.636116e-06
M_X        f_Clostridiaceae    1.521961e-07
Name: mass_rate*frequency, dtype: float64

#### L-Malate

**Receiver of L-Malate**

In [463]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Malate")].value_counts("family_receiver")

family_receiver
Other                 4
f_Clostridiaceae      2
f_Lactobacillaceae    2
dtype: int64

In [464]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Malate")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Clostridiaceae      0.003305
f_Lactobacillaceae    0.000476
Other                 0.000115
Name: mass_rate*frequency, dtype: float64

**Donor of L-Malate**

In [465]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Malate")].value_counts("family_donor")

family_donor
f_Lachnospiraceae    4
Other                2
f_Clostridiaceae     2
dtype: int64

In [466]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Malate")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
Other                0.003570
f_Lachnospiraceae    0.000256
f_Clostridiaceae     0.000070
Name: mass_rate*frequency, dtype: float64

In [467]:
steadier_sample_cross[(steadier_sample_cross.compound=="L-Malate")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor     
CD_X       f_Lachnospiraceae    0.000045
CM_X       Other                0.003570
           f_Lachnospiraceae    0.000211
M_X        f_Clostridiaceae     0.000070
Name: mass_rate*frequency, dtype: float64

#### Succinate

**Receiver of Succinate**

In [468]:
steadier_sample_cross[(steadier_sample_cross.compound=="Succinate")].value_counts("family_receiver")

family_receiver
f_Lactobacillaceae    1
dtype: int64

In [469]:
steadier_sample_cross[(steadier_sample_cross.compound=="Succinate")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Lactobacillaceae    0.000025
Name: mass_rate*frequency, dtype: float64

**Donor of Succinate**

In [470]:
steadier_sample_cross[(steadier_sample_cross.compound=="Succinate")].value_counts("family_donor")

family_donor
f_Clostridiaceae    1
dtype: int64

In [471]:
steadier_sample_cross[(steadier_sample_cross.compound=="Succinate")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Clostridiaceae    0.000025
Name: mass_rate*frequency, dtype: float64

In [472]:
steadier_sample_cross[(steadier_sample_cross.compound=="Succinate")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor    
CM_X       f_Clostridiaceae    0.000025
Name: mass_rate*frequency, dtype: float64

#### Acetate

**Receiver of Acetate**

In [473]:
steadier_sample_cross[(steadier_sample_cross.compound.str.contains("Acetate"))].value_counts("family_receiver")

Series([], dtype: int64)

In [474]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetate")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


Series([], Name: mass_rate*frequency, dtype: float64)

**Donor of Acetate**

In [475]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetate")].value_counts("family_donor")

Series([], dtype: int64)

In [476]:
steadier_sample_cross[(steadier_sample_cross.compound=="Acetate")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


Series([], Name: mass_rate*frequency, dtype: float64)